In [1]:
from typing import List, Set, Any, Tuple, Dict, Union
from collections.abc import Sequence

import random
from fuzzingbook.Coverage import population_coverage

In [2]:
class Mutator:
    def __init__(self) -> None:
        self.mutators = [
            self.delete_random_character,
            self.insert_random_character,
            self.flip_random_character,
        ]

In [ ]:
class Mutator(Mutator):
    def insert_random_character(self, s: str) -> str:
        pos = random.randint(0, len(s))
        random_character = chr(random.randrange(32, 127))
        return s[:pos] + random_character + s[pos:]

    def delete_random_character(self, s: str) -> str:
        if s == "":
            return self.insert_random_character(s)

        pos = random.randint(0, len(s) - 1)
        return s[:pos] + s[pos + 1 :]

    def flip_random_character(self, s: str) -> str:
        if s == "":
            return self.insert_random_character

        pos = random.randint(0, len(s) - 1)
        c = s[pos]
        bit = 1 << random.randint(0, 6)
        new_c = chr(ord(c) ^ bit)

        return s[:pos] + new_c + s[pos + 1 :]

    def mutate(self, inp: Any) -> Any:
        mutator = random.choice(self.mutators)
        return mutator(inp)


Mutator().mutate("good")

'gond'

In [ ]:
from fuzzingbook.Coverage import Location


class Seed:
    def __init__(self, data: str) -> None:
        self.data = data
        self.coverage: Set[Location] = set()
        self.distance: Union[int, float] = -1
        self.energy = 0.0

    def __str__(self) -> str:
        return self.data

    __repr__ = __str__

In [ ]:
class PowerSchedule:
    def __init__(self) -> None:
        self.path_frequency: Dict = {}

    def assignEnergy(self, population: Sequence[Seed]) -> None:
        for seed in population:
            seed.energy = 1

    def normalizedEnergy(self, population: Sequence[Seed]) -> List[float]:
        energy = list(map(lambda seed: seed.energy, population))
        sum_energy = sum(energy)
        assert sum_energy != 0
        norm_energy = list(map(lambda nrg: nrg / sum_energy, energy))
        return norm_energy

    def choose(self, population: Sequence[Seed]) -> Seed:
        self.assignEnergy(population)
        norm_energy = self.normalizedEnergy(population)
        seed: Seed = random.choices(population, weights=norm_energy)[0]
        return seed

In [ ]:
population = [Seed("A"), Seed("B"), Seed("C")]

schedule = PowerSchedule()

hits = {"A": 0, "B": 0, "C": 0}

for i in range(10000):
    seed = schedule.choose(population)
    hits[seed.data] += 1

In [12]:
hits

{'A': 3355, 'B': 3331, 'C': 3314}

In [ ]:
from fuzzingbook.MutationFuzzer import FunctionCoverageRunner, http_program


def crashme(s: str) -> None:
    if len(s) > 0 and s[0] == "b":
        if len(s) > 1 and s[1] == "a":
            if len(s) > 2 and s[2] == "d":
                if len(s) > 3 and s[3] == "!":
                    raise Exception()


crashme_runner = FunctionCoverageRunner(crashme)
crashme_runner.run("good")
list(crashme_runner.coverage())

[('run_function', 410), ('crashme', 4)]

In [ ]:
from fuzzingbook.Fuzzer import Fuzzer


class AdvancedMutationFuzzer(Fuzzer):
    def __init__(
        self, seeds: List[str], mutator: Mutator, schedule: PowerSchedule
    ) -> None:
        self.seeds = seeds
        self.mutator = mutator
        self.schedule = schedule
        self.inputs: List[str] = []
        self.reset()